In [121]:
from qdrant_client import QdrantClient, models

In [102]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [103]:
import numpy as np
from fastembed import TextEmbedding

query = 'I just discovered the course. Can I join now?'
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_handle)


# Embed the query (returns a list with one vector)
embeddings = list(embedding_model.embed(query))

#print(embeddings)
len(embeddings[0])
#min(embeddings[0])
array_size_512 = np.zeros(512)
array_size_512 = embeddings[0]
print(np.min(array_size_512))

-0.11726373022974847


In [104]:
import numpy as np
from fastembed import TextEmbedding
doc = 'Can I still join the course after the start date?'
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_handle)
doc_embeddings = list(embedding_model.embed(doc))
print(doc_embeddings[0])
#len(embeddings[0])
#min(embeddings[0])
#array_size_512 = np.zeros(512)
#array_size_512 = embeddings
#print(np.min(array_size_512)) 

[-5.45304133e-02 -7.83451779e-02  3.13610405e-02  2.34234822e-02
 -3.06321542e-02 -5.51041722e-02  2.24726469e-02  1.50386032e-02
 -1.81159762e-02  1.80595373e-02 -4.80890398e-03 -7.57395660e-02
 -4.87271651e-03 -9.07717901e-02  4.83940937e-02 -4.26317272e-02
 -2.61462449e-02 -4.48129083e-02 -2.06926031e-02 -3.79054511e-02
 -3.30329200e-02 -8.02849396e-03  1.18560490e-02 -3.83721080e-02
  7.26378360e-02 -5.82811751e-02 -7.30890342e-02  3.40911512e-02
  5.60705435e-02  5.02684926e-02 -5.99293455e-02  5.33016467e-02
 -1.79524656e-02  4.17742530e-03 -2.45427787e-02  1.32288183e-02
  4.64177968e-02  1.09963575e-02 -1.01499833e-02 -5.77523997e-03
 -2.83978587e-04  4.91496389e-02  1.11214839e-01 -2.76306232e-02
 -6.78923645e-02 -1.85928277e-02 -1.23963212e-01  1.25745928e-02
  1.65298537e-02  7.22241271e-03 -2.77237382e-02  5.18073706e-02
 -5.98604913e-02  5.27339588e-02  1.03844065e-02  4.82091147e-02
 -2.29023363e-02  1.58800078e-02 -4.80511541e-02  1.02489381e-02
  1.37884565e-02  3.87445

In [105]:
# Create collection
collection_name = "zoomcamp-hw2"
client.create_collection(collection_name=collection_name,vectors_config=models.VectorParams(size=512, distance=models.Distance.COSINE))

True

In [106]:
# Create points with 2 text obj (query and doc)
points = []
identifier = 0
docs = [query, doc]
for each in docs:
    #embedding = list(embedding_model.embed(each))[0]
    point = models.PointStruct(id=identifier, vector=models.Document(text=each, model=model_handle),payload={})
    points.append(point)
    identifier = identifier + 1
    
points

[PointStruct(id=0, vector=Document(text='I just discovered the course. Can I join now?', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={}),
 PointStruct(id=1, vector=Document(text='Can I still join the course after the start date?', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={})]

In [107]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [108]:
retrieved_point0 = client.retrieve(
        collection_name=collection_name,
        ids=[0], # Retrieve point with ID 0
        with_payload=False,
        with_vectors=True,
    )
retrieved_point0[0].vector


[-0.07639464,
 -0.07305555,
 0.05865017,
 0.039267052,
 -0.014179709,
 -0.04683428,
 0.026986282,
 0.03490392,
 0.0015342056,
 -0.011040754,
 0.055755626,
 -0.028836023,
 -0.048723962,
 -0.09909239,
 0.06025796,
 -0.018610543,
 -0.008810406,
 -0.030199492,
 -0.014254169,
 -0.038788427,
 -0.027612565,
 -0.0069495845,
 0.032065343,
 -0.0051783794,
 0.08379599,
 -0.08873297,
 -0.07302605,
 0.059284665,
 0.04078055,
 0.07726848,
 -0.046002954,
 0.03984268,
 0.017482154,
 0.008702663,
 -0.0309291,
 0.02204704,
 0.047647998,
 0.017656632,
 -0.036201373,
 -0.035340827,
 -0.0059253783,
 0.029709125,
 0.08058836,
 0.010713263,
 -0.053819254,
 -0.0096288,
 -0.11726373,
 0.03365464,
 0.007057629,
 0.007701601,
 -0.05174826,
 0.054695547,
 -0.07802899,
 0.04218017,
 0.012625102,
 0.053133912,
 -0.026777722,
 0.016882064,
 -0.037506454,
 0.009131969,
 0.021369688,
 0.016015017,
 -0.05056502,
 0.056220617,
 -0.03668027,
 -0.03854168,
 -0.042125713,
 -0.013692141,
 -0.08585819,
 0.02626408,
 -0.06185

In [156]:
retrieved_point1 = client.retrieve(
        collection_name=collection_name,
        ids=[1], # Retrieve point with ID 1
        with_payload=False,
        with_vectors=True,
    )
retrieved_point1
#retrieved_point1[0].vector

[Record(id=1, payload=None, vector=[-0.022364719, -0.02610956, 0.039368987, -0.010689968, 0.07897592, 0.041451383, 0.011153311, -0.021060454, 0.021172574, 0.017244214, -0.006062637, 0.015078318, 0.004280171, 0.027935656, -0.0070814425, -0.01547394, -0.011933198, -0.054536723, -0.040166255, 0.030011853, 0.008030139, 0.021405997, -0.02211349, -0.011304892, 0.0064406884, 0.019187085, 0.006093563, -0.0016964308, -0.01599324, -0.1487155, -0.03384478, -0.066284455, 0.0065491875, 0.01355843, 0.04744872, 0.0070717866, 0.00114966, 0.048640434, -0.030253211, 0.035635453, 0.026967967, -0.013829668, 0.004069123, -0.04292064, -0.030717911, -0.003378268, 0.0024660986, -0.030483246, 0.045287386, -0.012537971, 0.017365368, -0.028533198, 0.026791524, -0.017725408, -0.029336851, -0.023553832, 0.023026923, 0.012196632, 0.004077314, 0.019008704, 0.012773868, -0.013107991, -0.20518476, 0.07058533, -0.06237332, 0.039597973, -0.0015384723, -0.004274489, 0.02129877, 0.03876727, -0.05658178, 0.012557946, -0.02

In [110]:
similarity = np.dot(retrieved_point0[0].vector, retrieved_point1[0].vector)
print(similarity)

0.9008529102612799


In [111]:
import json

models = []
for model in TextEmbedding.list_supported_models():
    modelname, dim =  model["model"], model["dim"]
    models.append({"modelname": modelname, "dim": dim})

In [112]:
models

[{'modelname': 'BAAI/bge-base-en', 'dim': 768},
 {'modelname': 'BAAI/bge-base-en-v1.5', 'dim': 768},
 {'modelname': 'BAAI/bge-large-en-v1.5', 'dim': 1024},
 {'modelname': 'BAAI/bge-small-en', 'dim': 384},
 {'modelname': 'BAAI/bge-small-en-v1.5', 'dim': 384},
 {'modelname': 'BAAI/bge-small-zh-v1.5', 'dim': 512},
 {'modelname': 'mixedbread-ai/mxbai-embed-large-v1', 'dim': 1024},
 {'modelname': 'snowflake/snowflake-arctic-embed-xs', 'dim': 384},
 {'modelname': 'snowflake/snowflake-arctic-embed-s', 'dim': 384},
 {'modelname': 'snowflake/snowflake-arctic-embed-m', 'dim': 768},
 {'modelname': 'snowflake/snowflake-arctic-embed-m-long', 'dim': 768},
 {'modelname': 'snowflake/snowflake-arctic-embed-l', 'dim': 1024},
 {'modelname': 'jinaai/jina-clip-v1', 'dim': 768},
 {'modelname': 'Qdrant/clip-ViT-B-32-text', 'dim': 512},
 {'modelname': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384},
 {'modelname': 'jinaai/jina-embeddings-v2-base-en', 'dim': 768},
 {'modelname': 'jinaai/jina-embeddings-v

In [113]:
import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] < EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

In [114]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [115]:
documents

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [118]:
len(documents)

5

In [122]:
points = []
id = 3

for doc in documents:
    #print(doc)

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [123]:
points

[PointStruct(id=3, vector=Document(text="Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'course': 'data-engineering-zoomcamp'}),
 PointStruct(id=4, vector=Document(text='Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', model='jinaai/jina-embeddings-v2-small

In [124]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [132]:
pts = [3,4,5,6,7]
res = []
for pt in pts:
    current_retrieved_point = client.retrieve(
        collection_name=collection_name,
        ids=[pt], # Retrieve point with ID pt
        with_payload=False,
        with_vectors=True,
    )
    #print(retrieved_point0[0].vector)
    #print(current_retrieved_point[0].vector)
    print(f"Fetched pt: {pt} current_retrieved_point {current_retrieved_point[0].vector}")
    similarityQ3 = np.dot(retrieved_point0[0].vector, current_retrieved_point[0].vector)
    res.append(similarityQ3)
    

Fetched pt: 3 current_retrieved_point [-0.024952471, -0.039645385, -0.004376747, 0.02958303, -0.012030076, -0.0069325007, -7.656725e-05, 0.06151213, -0.028510582, -0.06709609, 0.0065319235, -0.05095759, -0.05012441, -0.058327634, 0.0954579, -0.05410399, -0.032038596, -0.046956033, -0.0616868, -0.09039345, -0.033534914, -0.029053388, 0.044497225, -0.07000456, 0.042759545, -0.07019891, -0.08680373, 0.08142058, 0.07218799, 0.01894729, -0.059557684, 0.010949329, -0.018051147, 0.046081778, -0.029758869, 0.034579217, 0.040522154, 0.017594993, -0.027704153, -0.016721169, -0.05028602, 0.013497159, 0.09174102, 0.008328603, -0.07495525, 0.0023509024, -0.04665368, 0.012956048, 0.02577389, 0.005871563, -0.047506247, 0.0141338315, -0.10499458, 0.031179463, -0.029514957, 0.06340931, -0.012428459, 0.029035192, -0.024678705, 0.0065530515, -0.001689952, 0.04806843, -0.0451916, 0.06588197, 0.0007003945, -0.046943318, -0.0064393315, -0.022500765, -0.11023053, 0.03249054, -0.10615784, 0.030156547, 0.06316

In [124]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [133]:
print(res)

[np.float64(0.7629684545728064), np.float64(0.8182378183427668), np.float64(0.8085397347508796), np.float64(0.7133078685570795), np.float64(0.7304499266693546)]


In [137]:
#Q3
# query = 'I just discovered the course. Can I join now?'
# text='Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.
res[1]

np.float64(0.8182378183427668)

In [138]:
#Q4 Now let's calculate a new field, which is a concatenation of question and text:
# full_text = doc['question'] + ' ' + doc['text']
# Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?


# Q4 create

points = []
id = 8

for doc in documents:
    #print(doc)
    full_text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=full_text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [139]:
# Q4 upsert

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [140]:
# Q4 retrieve

pts = [8,9,10,11,12]
res = []
for pt in pts:
    current_retrieved_point = client.retrieve(
        collection_name=collection_name,
        ids=[pt], # Retrieve point with ID pt
        with_payload=False,
        with_vectors=True,
    )
    #print(retrieved_point0[0].vector)
    #print(current_retrieved_point[0].vector)
    print(f"Fetched pt: {pt} current_retrieved_point {current_retrieved_point[0].vector}")
    similarityQ4 = np.dot(retrieved_point0[0].vector, current_retrieved_point[0].vector)
    res.append(similarityQ4)

Fetched pt: 8 current_retrieved_point [-0.039587475, -0.0690407, 0.01146618, 0.026260396, -0.0247341, -0.024720693, 0.003439709, 0.042785924, -0.024162944, -0.026199237, -0.014921459, -0.07881787, -0.024740098, -0.07740145, 0.07686429, -0.046568375, -0.030845456, -0.044565722, -0.04481413, -0.060655624, -0.036711335, -0.018000936, 0.031952284, -0.06805735, 0.059340086, -0.0710709, -0.08380715, 0.07134353, 0.07235555, 0.024347816, -0.055102453, 0.02945919, -0.018308265, 0.025609974, -0.031728674, 0.027675094, 0.04963585, 0.013883948, -0.020100309, -0.0028251538, -0.029371914, 0.028263265, 0.100648426, 0.0028943631, -0.07165614, 0.00076785556, -0.08024243, 0.011240777, 0.0255504, 0.015388169, -0.03343803, 0.02859502, -0.07989595, 0.034472, -0.007653952, 0.0663229, -0.010571317, 0.024312276, -0.040355776, -0.0028037447, -0.0017486297, 0.030156884, -0.05380364, 0.055337228, -0.010549605, -0.036919657, -0.030680662, -0.027392665, -0.10284295, 0.016560763, -0.09235248, 0.019417338, 0.0747836

In [142]:
print(res)

[np.float64(0.851454330199421), np.float64(0.8436594138456714), np.float64(0.8408287039703563), np.float64(0.7755157971735214), np.float64(0.8086007988045588)]


In [148]:
#Q6 use BAAI/bge-small-en

In [181]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [182]:
#list of dictionary
# len(documents) - 375
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [183]:
# Create collection
collection_name = "zoomcamp-hw2-q6v2"
client.create_collection(collection_name=collection_name,vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE))

True

In [184]:
model_handle = "BAAI/bge-small-en"
# Create Points from documents
# text = doc['question'] + ' ' + doc['text']

points = []
id = 0

for doc in documents:
    #print(doc)
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle), #embed text locally with "BAAI/bge-small-en" from FastEmbed
        payload={
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

#points

In [185]:
# Upsert points to collection - 375 points
client.upsert(
    collection_name=collection_name,
    points=points
)



UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [157]:
# Add a Q1 question to embed and generate cosine similarity
# Points with id 0-374 are already added.

query = 'I just discovered the course. Can I join now?'

# Add query as a point (id = 375)

points = []
identifier = 375
docs = [query]
for each in docs:
    point = models.PointStruct(id=identifier, vector=models.Document(text=each, model=model_handle),payload={})
    points.append(point)
    identifier = identifier + 1
    
points

[PointStruct(id=375, vector=Document(text='I just discovered the course. Can I join now?', model='BAAI/bge-small-en', options=None), payload={})]

In [158]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [168]:
# Retrieve 375 point
pt = 375
retrieved_point375 = client.retrieve(
        collection_name=collection_name,
        ids=[pt], # Retrieve point with ID pt
        with_payload=False,
        with_vectors=True,
)
retrieved_point375[0].vector

[-0.054452203,
 -0.068776846,
 -0.0029984291,
 -0.0406731,
 0.01692194,
 0.006487891,
 0.023626829,
 -0.0005644122,
 0.0013558328,
 -0.0293422,
 0.035698224,
 -0.0628751,
 0.01950473,
 0.018992705,
 0.03750916,
 0.003805018,
 -0.017636338,
 -0.018257802,
 -0.035816077,
 0.031308234,
 0.002387627,
 -0.01286112,
 -0.015096147,
 -0.024604484,
 -0.0027202927,
 0.05598972,
 0.002047138,
 -0.0236369,
 -0.032707106,
 -0.12462115,
 -0.022795996,
 -0.03243251,
 0.0064037414,
 0.04113868,
 0.030957723,
 -0.021314202,
 -0.023365196,
 0.018322406,
 -0.01519472,
 0.05845548,
 0.035400946,
 -0.00073003076,
 -0.0011920969,
 -0.01885079,
 0.009262446,
 -0.043694112,
 -0.014536778,
 0.013849587,
 0.07851348,
 -0.068936944,
 0.0027946082,
 -0.059111904,
 -0.019268166,
 0.012492726,
 0.0042150747,
 0.06749926,
 0.036519118,
 0.023628656,
 -0.0053993426,
 0.039924074,
 0.02723557,
 0.020830916,
 -0.20958477,
 0.0748936,
 -0.04233175,
 0.015821608,
 -0.00055187347,
 -0.028550103,
 0.046688717,
 0.003329957

In [167]:
# Loop over range 0 - 375 and compute dot product and store result in res 
pts = range(0, 375)
res = []
for pt in pts:
    current_retrieved_point = client.retrieve(
        collection_name=collection_name,
        ids=[pt], # Retrieve point with ID pt
        with_payload=False,
        with_vectors=True,
    )
    similarityQ6 = np.dot(retrieved_point375[0].vector, current_retrieved_point[0].vector)
    res.append(similarityQ6)

print(res)

[np.float64(0.8507795033537302), np.float64(0.7428698839055344), np.float64(0.7564047597653082), np.float64(0.7950476526034281), np.float64(0.8072042374635754), np.float64(0.8248839334559899), np.float64(0.7520699882655216), np.float64(0.7730481508313906), np.float64(0.7195788676742095), np.float64(0.7652492215754126), np.float64(0.7802565304119893), np.float64(0.8660218963333388), np.float64(0.9999999073574155), np.float64(0.7976436412687707), np.float64(0.7343812370455853), np.float64(0.7668941641547649), np.float64(0.7098851369340557), np.float64(0.8288912621351324), np.float64(0.784616589395328), np.float64(0.742526392022795), np.float64(0.7857192036970336), np.float64(0.778316069209932), np.float64(0.7908901344930226), np.float64(0.791337213325833), np.float64(0.7855506823288052), np.float64(0.7524868043894852), np.float64(0.7813317809768292), np.float64(0.8022162411384601), np.float64(0.7363953407404995), np.float64(0.8345984401341836), np.float64(0.7717107179000149), np.float64(

In [170]:
np.max(res)

np.float64(0.9999999073574155)

In [171]:
np.sort(res)

array([0.68225102, 0.69607077, 0.70988514, 0.71957887, 0.72574953,
       0.72977416, 0.73097582, 0.73257995, 0.73376362, 0.73400863,
       0.73438124, 0.73564457, 0.7356735 , 0.73608202, 0.73639534,
       0.73669336, 0.73688894, 0.7397949 , 0.74020958, 0.74034765,
       0.74252639, 0.74286988, 0.74334083, 0.74350174, 0.74422927,
       0.74470831, 0.74551434, 0.74615709, 0.74622481, 0.7467202 ,
       0.74765074, 0.74801245, 0.74826799, 0.74889125, 0.7489438 ,
       0.74988761, 0.75091281, 0.75095271, 0.75154511, 0.75160582,
       0.75206999, 0.7524868 , 0.75249201, 0.75280857, 0.75440954,
       0.75479134, 0.75513694, 0.75522243, 0.75536144, 0.7553742 ,
       0.75594245, 0.75612893, 0.75640476, 0.75644899, 0.75655014,
       0.75764297, 0.75768469, 0.75922234, 0.75966216, 0.75993297,
       0.76012906, 0.76030526, 0.76056432, 0.76061595, 0.7608335 ,
       0.76136793, 0.76151107, 0.76180554, 0.762741  , 0.76285771,
       0.76286089, 0.76299182, 0.7631225 , 0.76327745, 0.76382

In [176]:
def search(collection_name, query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [186]:
results = search("zoomcamp-hw2-q6v2", "I just discovered the course. Can I join now?", 5)

In [187]:
results

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703173, payload={'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.8691884, payload={'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=13, version=0, score=0.86833125, payload={'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=293, version=0, score=0.8576106, payload={'section': '10. Kubernetes and TensorFlow Serving', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=17, version=0, score=0.85715395, payload={'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])